<a href="https://colab.research.google.com/github/2003Yash/layer-normalization/blob/main/Layer_Normalization_for_Transformer_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We get a word embedding vector from multi-head attention block and each head will give a vector now we will add the all attention head vector intoa single vector and we also has a residual connection from the original word embedding ( so that we don't vary much from the original word or the original context ) and we perform this normalization so that the learnable prams in neural will not flutuate too much in the back propagation updation phase


first we get a matrix which is added version of attention vectors from all the attention heads and we find mean and s.d for all rows and then we use  ->  formula (x-mean)/s.d for all values in matrix
-> then norm formula = alpha*Y + b (Here alpha and beta are learnable params and alpha is multiplied withh all y values ) the matrix resultant after this process is normalized matrix

In [ ]:
import tensorflow as tf

class LayerNormalization(tf.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        self.parameters_shape = parameters_shape
        self.eps = eps
        self.gamma = tf.Variable(tf.ones(parameters_shape), trainable=True)
        self.beta = tf.Variable(tf.zeros(parameters_shape), trainable=True)

# The whole process is explained in the 2nd text cell in this file

    def __call__(self, inputs):
        dims = [-i for i in range(1, len(self.parameters_shape) + 1)]
        mean = tf.reduce_mean(inputs, axis=dims, keepdims=True)
        print(f"Mean \n ({mean.shape}): \n {mean}")
        var = tf.reduce_mean(tf.square(inputs - mean), axis=dims, keepdims=True)
        std = tf.sqrt(var + self.eps)
        print(f"Standard Deviation \n ({std.shape}): \n {std}")
        y = (inputs - mean) / std
        print(f"y \n ({y.shape}) = \n {y}")
        out = self.gamma * y + self.beta
        print(f"out \n ({out.shape}) = \n {out}")
        return out

In [ ]:
batch_size = 3
sentence_length = 5
embedding_dim = 8
inputs = tf.random.normal(shape=(sentence_length, batch_size, embedding_dim))

In [ ]:
print(f"input \n ({inputs.shape}) = \n {inputs}")

input 
 ((5, 3, 8)) = 
 [[[-1.0038837e+00 -9.5244807e-01 -1.6853817e+00 -2.7049094e-01
    1.3179457e-01 -1.9687043e+00 -1.9520327e-01  2.4850056e-01]
  [-1.5948204e+00  6.2333506e-01  9.7797382e-01  2.9508597e-01
    1.1022136e+00 -5.3846568e-01  1.9445401e-01 -7.3187083e-01]
  [ 7.6458985e-01 -1.5157856e-01 -2.5566173e+00  1.1122432e-01
    8.3853531e-01  1.3742457e-01  6.1803114e-01 -4.6780297e-01]]

 [[-5.3838140e-01  3.8508824e-01  1.0166631e+00  2.0144999e+00
   -8.0260873e-01  6.2671983e-01 -9.8531656e-02 -4.7026455e-01]
  [-3.0046874e-01 -1.3527232e-02  1.2161305e+00 -4.0085322e-01
   -1.3490324e+00  6.7868131e-01  1.3866105e+00 -6.8107778e-01]
  [ 7.2391754e-01  4.8476183e-01 -5.9137559e-01  1.2990150e+00
   -2.6015815e-01 -3.3298323e-01  5.9793252e-01  1.4928647e+00]]

 [[ 6.3440102e-01  4.4141170e-01  3.0047151e-01 -2.7355605e-01
   -1.1412312e+00  6.9188708e-01 -9.7919688e-02  2.8614568e-02]
  [ 1.3173200e+00 -4.3998680e-01 -3.8386998e-01 -1.4733223e+00
   -1.8362412e+00 -1

In [ ]:
layer_norm = LayerNormalization(inputs.shape[-1:])

In [ ]:
out = layer_norm(inputs)

Mean 
 ((5, 3, 1)): 
 [[[-0.7119771 ]
  [ 0.04098821]
  [-0.08827421]]

 [[ 0.26664808]
  [ 0.06705788]
  [ 0.42674685]]

 [[ 0.07300987]
  [-0.8206995 ]
  [ 0.07919765]]

 [[-0.8401872 ]
  [ 0.17514521]
  [-0.4841744 ]]

 [[ 0.08533464]
  [ 0.27161366]
  [ 0.1214119 ]]]
Standard Deviation 
 ((5, 3, 1)): 
 [[[0.7718832 ]
  [0.86893713]
  [1.0257196 ]]

 [[0.88240063]
  [0.89171743]
  [0.7162508 ]]

 [[0.5602884 ]
  [1.0376303 ]
  [1.0992057 ]]

 [[0.9388038 ]
  [0.84947455]
  [1.0166148 ]]

 [[0.46725726]
  [0.58233607]
  [0.36655864]]]
y 
 ((5, 3, 8)) = 
 [[[-0.37817457 -0.311538   -1.2610775   0.57195985  1.0931339
   -1.6281314   0.66949743  1.2443304 ]
  [-1.8825396   0.67018294  1.0783123   0.29242364  1.2212913
   -0.6668536   0.17661324 -0.88943034]
  [ 0.8314787  -0.06171701 -2.40645     0.19449617  0.90357006
    0.22003944  0.688595   -0.3700122 ]]

 [[-0.91231745  0.13422492  0.84997106  1.9807917  -1.2117589
    0.40805927 -0.413848   -0.83512247]
  [-0.41215593 -0.09037068

In [ ]:
tf.print(out[0].numpy().mean(), out[0].numpy().std())

-1.9868216e-08 0.99999344
